In [1]:
import pandas as pd
import numpy as np
import cv2
import os

import tensorflow as tf
from tensorflow.keras import models, layers, datasets

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import seaborn as sns

from plotting import show_distribution
from plot_val import calc_roc_curve, plot_roc_curve, plot_confusion_matrix

from apply_processing import apply, resize_images, function_gray, threshold_images
from apply_processing import function_label, blur_images, adjust_contrast_images, rotated_labels
from apply_processing import filter_images, edge_images, contour_images, rotate_images

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(f"Nums GPUs available : {len(physical_devices)}")

tf.config.experimental.set_memory_growth(physical_devices[0], True)

Nums GPUs available : 1


In [3]:
df = pd.read_csv("Dataset\\images_mushrooms.csv")

df.head()

,Image,Label
0,Dataset\Mushrooms\Agaricus\009_mrv34Sn4WiQ.jpg,Agaricus
1,Dataset\Mushrooms\Agaricus\010_Rk1D3EG856g.jpg,Agaricus
2,Dataset\Mushrooms\Agaricus\017_QHGfbEiWvq8.jpg,Agaricus
3,Dataset\Mushrooms\Agaricus\020_vZJfrjIwmqU.jpg,Agaricus
4,Dataset\Mushrooms\Agaricus\030_-i6kYqDvBQo.jpg,Agaricus


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Image   900 non-null    object
 1   Label   900 non-null    object
dtypes: object(2)
memory usage: 14.2+ KB


In [5]:
df['Label'].value_counts() / df.shape[0]

Agaricus       0.111111
Amanita        0.111111
Suillus        0.111111
Russula        0.111111
Boletus        0.111111
Entoloma       0.111111
Cortinarius    0.111111
Lactarius      0.111111
Hygrocybe      0.111111
Name: Label, dtype: float64

# Processing the images

In [6]:
# Canny + fixed contrast & brightness

def processing_canny(df):
    images = df['Image']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 20
    
    mod_images = function_gray(images)
    mod_images = resize_images(mod_images, size)
    mod_images = filter_images(mod_images)
    mod_images = adjust_contrast_images(mod_images, contrast=contrast, brightness=brightness)
    mod_images = blur_images(mod_images)
    mod_images = edge_images(mod_images, edge=cv2.Canny)
    
    return list(mod_images)

In [7]:
# Threshold otsu + fixed contrast & brightness

def processing_threshold(df):
    images = df['Image']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 20
    
    mod_images = function_gray(images)
    mod_images = resize_images(mod_images, size)
    mod_images = filter_images(mod_images)
    mod_images = adjust_contrast_images(mod_images, contrast=contrast, brightness=brightness)
    mod_images = blur_images(mod_images)
    
    mod_images = threshold_images(mod_images, threshold_type=cv2.THRESH_BINARY-cv2.THRESH_OTSU)
    
    return list(mod_images)

In [8]:
# Laplacian + fixed contrast & brightness

def processing_laplacian(df):
    images = df['Image']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 20
    
    mod_images = function_gray(images)
    mod_images = resize_images(mod_images, size)
    mod_images = filter_images(mod_images)
    mod_images = adjust_contrast_images(mod_images, contrast=contrast, brightness=brightness)
    mod_images = blur_images(mod_images)
    mod_images = edge_images(mod_images, edge=cv2.Laplacian, ddepth=50)
    
    return list(mod_images)

In [9]:
# Sobelx + fixed contrast & brightness

def processing_sobelx(df):
    images = df['Image']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 20
    
    mod_images = function_gray(images)
    mod_images = resize_images(mod_images, size)
    mod_images = filter_images(mod_images)
    mod_images = adjust_contrast_images(mod_images, contrast=contrast, brightness=brightness)
    mod_images = blur_images(mod_images)
    mod_images = edge_images(mod_images, edge=cv2.Sobel, dx=1)
    
    return list(mod_images)

In [10]:
# Sobely + fixed contrast & brightness

def processing_sobely(df):
    images = df['Image']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 20
    
    mod_images = function_gray(images)
    mod_images = resize_images(mod_images, size)
    mod_images = filter_images(mod_images)
    mod_images = adjust_contrast_images(mod_images, contrast=contrast, brightness=brightness)
    mod_images = blur_images(mod_images)
    mod_images = edge_images(mod_images, edge=cv2.Sobel, dy=1)
    
    return list(mod_images)

In [11]:
# Sobelxy + fixed contrast & brightness

def processing_sobelxy(df):
    images = df['Image']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 20
    
    mod_images = function_gray(images)
    mod_images = resize_images(mod_images, size)
    mod_images = filter_images(mod_images)
    mod_images = adjust_contrast_images(mod_images, contrast=contrast, brightness=brightness)
    mod_images = blur_images(mod_images)
    mod_images = edge_images(mod_images, edge=cv2.Sobel, dx=1, dy=1)
    
    return list(mod_images)

In [12]:
# Fixed contrast & brightness

def processing_fixed(df):
    images = df['Image']
    
    size = (150, 150)
    contrast = 1.75
    brightness = 20
    
    mod_images = function_gray(images)
    mod_images = resize_images(mod_images, size)
    mod_images = filter_images(mod_images)
    mod_images = adjust_contrast_images(mod_images, contrast=contrast, brightness=brightness)
    mod_images = blur_images(mod_images)
    
    return list(mod_images)

In [13]:
# Random contrast & brightness

def processing_random(df):
    images = df['Image']
    
    size = (150, 150)
    contrast = np.random.randint(1, 5)
    brightness = np.random.randint(-100, 100)
    
    mod_images = function_gray(images)
    mod_images = resize_images(mod_images, size)
    mod_images = filter_images(mod_images)
    mod_images = adjust_contrast_images(mod_images, contrast=contrast, brightness=brightness)
    mod_images = blur_images(mod_images)
    
    return list(mod_images)

In [14]:
functions = [processing_canny, processing_threshold, processing_laplacian, 
             processing_sobelx, processing_sobely, processing_sobelxy, 
             processing_fixed, processing_random]

labels = [function_label, function_label, function_label, 
          function_label, function_label, function_label, 
          function_label, function_label]

images_aug, labels_aug = apply(functions, labels, df)

C:\Users\Bina Umat\Desktop\Python\Data Science\Kaggle Dataset\Mushrooms Classification\Computer Vision\apply_processing.py:157: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  thresh = np.array(thresh_images)[:, 0]
C:\Users\Bina Umat\Desktop\Python\Data Science\Kaggle Dataset\Mushrooms Classification\Computer Vision\apply_processing.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mod_images = np.array(thresh_images)[:, 1]


# Images info

In [15]:
print(f"Length of images no augmentation {len(images_aug)}")
print(f"Length of labels no augmentation {len(labels_aug)}")

Length of images no augmentation 7200
Length of labels no augmentation 7200


## Distribution

In [16]:
# show_distribution(images_noAug)

# Modelling

In [17]:
# le = LabelEncoder()
# le.fit(labels)

In [18]:
# labels = le.transform(labels_noAug)
# images = images_noAug / 255.0

In [19]:
# X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.33, random_state=42)

In [20]:
# model = models.Sequential()
# model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(150, 150, 1)))
# model.add(layers.MaxPooling2D((3,3)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((3,3)))
# model.add(layers.Conv2D(32, (3,3), activation='relu'))

In [21]:
# model.summary()

In [22]:
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='sigmoid'))
# model.add(layers.Dense(4))

In [23]:
# model.summary()

In [24]:
# adam_callback = tf.keras.callbacks.TensorBoard(log_dir='CNN_board/adam', histogram_freq=1)
# sgd_callback = tf.keras.callbacks.TensorBoard(log_dir='CNN_board/sgd', histogram_freq=1)

# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [25]:
# model_training = model.fit(X_train, y_train, epochs=6, 
#                            validation_data=(X_test, y_test), batch_size=32)

In [26]:
# model.evaluate(X_test, y_test)

## Evaluation

### ROC and AUC

In [27]:
# model_proba = model.predict(X_test)

In [28]:
# fprs, tprs, aucs = calc_roc_curve(y_test, model_proba)

In [29]:
# fig, ax = plt.subplots(figsize=(15,9))

# plot_roc_curve(fprs, tprs, aucs, "CNN")

### Confusion Matrix

In [30]:
# model_pred = np.argmax(model.predict(X_test), axis=1)

In [31]:
# fig, ax = plt.subplots(figsize=(15, 9))

# plot_confusion_matrix(le.inverse_transform(labels), y_test, model_pred, 'CNN')